In [4]:
from openai import OpenAI
client = OpenAI()
import json
import pandas as pd

In [61]:

batch_input_file = client.files.create(
  file=open("commands/batchinput.jsonl", "rb"),
  purpose="batch"
)

In [62]:
batch_input_file_id = batch_input_file.id

nesto = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

batch_input_file_id

'file-4zkl7lVGXUzZ5gvYJnXXpfRa'

In [63]:
nesto

Batch(id='batch_Z9hM4ER1TLmPWM0fXkWSZUBr', completion_window='24h', created_at=1724315557, endpoint='/v1/chat/completions', input_file_id='file-4zkl7lVGXUzZ5gvYJnXXpfRa', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1724401957, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [64]:
nesto.id

'batch_Z9hM4ER1TLmPWM0fXkWSZUBr'

In [65]:
nesto.output_file_id

In [66]:
batch_job = client.batches.retrieve(nesto.id)


In [75]:
batch_job.status

'completed'

In [67]:
batch_job.output_file_id

'file-gbr6B7dAJfsIkQdC0EQWKn0X'

In [52]:
file_list = client.files.list()
file_list

SyncPage[FileObject](data=[FileObject(id='file-hB8l332rrbox2I7UwC64slXS', bytes=638, created_at=1724314004, filename='batch_Dil24G7j9Tfx5kYSHsAgoTTu_error.jsonl', object='file', purpose='batch_output', status='processed', status_details=None), FileObject(id='file-4boNepsCWA7tR0EZcAlu8ZV9', bytes=604, created_at=1724313999, filename='batchinput.jsonl', object='file', purpose='batch', status='processed', status_details=None), FileObject(id='file-LdeD1rqnl5NKinD6ruB2OKwB', bytes=1179, created_at=1724313204, filename='batch_moIvyRlt4y7RPuXB5jbf1dc9_output.jsonl', object='file', purpose='batch_output', status='processed', status_details=None), FileObject(id='file-9hk6bYhnNaqjFNU9CGiRvEZJ', bytes=516, created_at=1724313194, filename='batchinput.jsonl', object='file', purpose='batch', status='processed', status_details=None), FileObject(id='file-xYbTaC9zM8gQLB6gzXRP6HqB', bytes=1192, created_at=1724312260, filename='batch_5wV532FUKDGnWqgFdhnkjZPS_output.jsonl', object='file', purpose='batch_o

In [53]:
file_list.data

[FileObject(id='file-hB8l332rrbox2I7UwC64slXS', bytes=638, created_at=1724314004, filename='batch_Dil24G7j9Tfx5kYSHsAgoTTu_error.jsonl', object='file', purpose='batch_output', status='processed', status_details=None),
 FileObject(id='file-4boNepsCWA7tR0EZcAlu8ZV9', bytes=604, created_at=1724313999, filename='batchinput.jsonl', object='file', purpose='batch', status='processed', status_details=None),
 FileObject(id='file-LdeD1rqnl5NKinD6ruB2OKwB', bytes=1179, created_at=1724313204, filename='batch_moIvyRlt4y7RPuXB5jbf1dc9_output.jsonl', object='file', purpose='batch_output', status='processed', status_details=None),
 FileObject(id='file-9hk6bYhnNaqjFNU9CGiRvEZJ', bytes=516, created_at=1724313194, filename='batchinput.jsonl', object='file', purpose='batch', status='processed', status_details=None),
 FileObject(id='file-xYbTaC9zM8gQLB6gzXRP6HqB', bytes=1192, created_at=1724312260, filename='batch_5wV532FUKDGnWqgFdhnkjZPS_output.jsonl', object='file', purpose='batch_output', status='proces

In [54]:
for file_info in file_list.data:
    if nesto.id in file_info.filename:
        print(file_info.id)
    # if file_info["filename"] == "your_file_name.txt":
    #     file_id = file_info["id"]
    #     print(f"File ID: {file_id}")
    #     break

file-hB8l332rrbox2I7UwC64slXS


In [68]:
file_response = client.files.content(batch_job.output_file_id)
print(file_response.text)
print(len(file_response.text.split("\n")))
for line in file_response.text.split("\n"):
    if line:
        obj = json.loads(line)
        print(obj)

{"id": "batch_req_up5WRFQzNWqInYkmol6LaLkj", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "fc20b91593553b798db59c48075bd41e", "body": {"id": "chatcmpl-9yxXDPpBTpPmuiK1fq8MTEBpdw84o", "object": "chat.completion", "created": 1724315559, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "Hello! How can I assist you today?", "refusal": null}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 20, "completion_tokens": 9, "total_tokens": 29}, "system_fingerprint": null}}, "error": null}
{"id": "batch_req_zxEKWQeVXM7plhCrIEDCqOiI", "custom_id": "request-2", "response": {"status_code": 200, "request_id": "2268077e22d32426f9259abd54efef51", "body": {"id": "chatcmpl-9yxXEyTkiGoDCutnKi7zEgHxbQB8z", "object": "chat.completion", "created": 1724315560, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "Greetings. How may I not assist you today?", "refus

In [70]:
res = client.files.content(batch_job.output_file_id).content

In [71]:
result_file_name = "commands/batch_job_results_movies.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(res)

In [72]:
# Loading data from saved file
results = []
with open(result_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [73]:
results

[{'id': 'batch_req_up5WRFQzNWqInYkmol6LaLkj',
  'custom_id': 'request-1',
  'response': {'status_code': 200,
   'request_id': 'fc20b91593553b798db59c48075bd41e',
   'body': {'id': 'chatcmpl-9yxXDPpBTpPmuiK1fq8MTEBpdw84o',
    'object': 'chat.completion',
    'created': 1724315559,
    'model': 'gpt-3.5-turbo-0125',
    'choices': [{'index': 0,
      'message': {'role': 'assistant',
       'content': 'Hello! How can I assist you today?',
       'refusal': None},
      'logprobs': None,
      'finish_reason': 'stop'}],
    'usage': {'prompt_tokens': 20, 'completion_tokens': 9, 'total_tokens': 29},
    'system_fingerprint': None}},
  'error': None},
 {'id': 'batch_req_zxEKWQeVXM7plhCrIEDCqOiI',
  'custom_id': 'request-2',
  'response': {'status_code': 200,
   'request_id': '2268077e22d32426f9259abd54efef51',
   'body': {'id': 'chatcmpl-9yxXEyTkiGoDCutnKi7zEgHxbQB8z',
    'object': 'chat.completion',
    'created': 1724315560,
    'model': 'gpt-3.5-turbo-0125',
    'choices': [{'index': 0,

In [83]:
from pathlib import Path
from typing import List, Dict


def save_failed_ids(failed: List[Dict[str, str]], dataset_name: str) -> None:
    file_path = Path(f"failed/failed_{dataset_name}.json")
    file_path.parent.mkdir(parents=True, exist_ok=True)
    # Write the IDs to a text file, one per line
    with open(file_path, "w") as f:
        # Convert exceptions to string because exceptions are not JSON serializable
        json.dump(failed, f, default=str, indent=4)

def make_dataset(
    processed_commands_path: Path, dataset_name: str
) -> Dict[str, str | List[str]]:
    returned_dict = {
        "id": [],
        "query": [],
        "passage_text": [],
    }
    failed = []
    # Open and iterate through the .jsonl file
    with open(processed_commands_path, "r") as file:
        for line in file:
            id_ = None
            print(line)
            try:
                data = json.loads(line)
                id_ = data["custom_id"].replace("task-", "")
                returned_data = data['response']['body']['choices'][0]['message']['content']
                tranlation = json.loads(
                    returned_data
                )  # gpt message i.e. translation in this case
                returned_dict["id"].append(id_)
                returned_dict["query"].append(tranlation["query"])
                returned_dict["passage_text"].append(tranlation["passage_text"])
            except Exception as e:
                failed.append({"id": id_, "exception": e})
    if failed:
        save_failed_ids(failed, dataset_name=dataset_name)
    return returned_dict

In [84]:
result_file_name = "commands/results_msmarco.jsonl"

a = make_dataset(processed_commands_path=result_file_name, dataset_name='msmarco')
a

{"id": "batch_req_st8Ee5AZ9Z8IouuXaa6O9ZzB", "custom_id": "task-0", "response": {"status_code": 200, "request_id": "bc8e8afe52b6f73bd609691b47eefd99", "body": {"id": "chatcmpl-9yy7fLf2PpHJxAg7Y30xUps2MbClh", "object": "chat.completion", "created": 1724317819, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n    \"query\": \"da li ljudska kosa zaustavlja veverice\",\n    \"passage_text\": [\"Hranimo veverice u na\u0161em dvori\u0161tu tokom jeseni i zime i primetili smo da neke od njih imaju \u0107elave fleke. Jedna ima nedostatak krzna du\u017e le\u0111a i ispod oba pazuha. Tako\u0111e, druga ima nekoliko mesta bez krzna na celom grudnom ko\u0161u. Sve one jedu i \u010dini se da imaju dobar apetit.\", \"\u017divotinje ne podnose miris ljudske kose, pa posipanje barijere od odse\u010denih dlaka oko va\u0161e ba\u0161te, ili lagano udubljivanje u zemlju prilikom sadnje lukovica, o\u010digledno ima neku vrednost. \u010citava situacija

{'id': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 'query': ['da li ljudska kosa zaustavlja veverice',
  'Koji su benefiti fosilnih goriva?',
  'šta je apotel',
  'prosečna cena za prilagođeni baldahin',
  'šta je hardver na računaru',
  'definicija logsitike EDI',
  'zašto bi trebalo da bude ilegalno rekreativno korišćenje marihuane',
  'U koju klasu spadaju paukovi?',
  'gde se nalazi amurski leopard',
  'koliko košta da se sterilizuje moja ženka kunića'],
 'passage_text': [['Hranimo veverice u našem dvorištu tokom jeseni i zime i primetili smo da neke od njih imaju ćelave fleke. Jedna ima nedostatak krzna duž leđa i ispod oba pazuha. Takođe, druga ima nekoliko mesta bez krzna na celom grudnom košu. Sve one jedu i čini se da imaju dobar apetit.',
   'Životinje ne podnose miris ljudske kose, pa posipanje barijere od odsečenih dlaka oko vaše bašte, ili lagano udubljivanje u zemlju prilikom sadnje lukovica, očigledno ima neku vrednost. Čitava situacija me pomalo nasmejava. Nika

## Checking tanslation results

In [47]:
import pandas as pd
pd.set_option('display.max_colwidth', 300)

msmarco_path = "datasets/msmarco.parquet"
pd.read_parquet(msmarco_path, engine='fastparquet')

,id,query,passage_text
0,1,Koji su benefiti fosilnih goriva?,[Najveća prednost korišćenja fosilnih goriva je ta što se lako mogu skladištiti i transportovati sa jednog mesta na drugo. Velike rezerve uglja se stoga prenose iz rudnika uglja do industrijskih postrojenja koja su udaljena od rudnika. Nafta se takođe prenosi do udaljenih elektrana kako bi se pr...
1,2,šta je apotema,"[Apotema. Apotema pravilnog mnogougla je segment nacrtan od centra mnogougla (tj. centra kruga koji opisuje mnogougao) do jedne strane, tako da je normalan na stranu. Ako pravilni mnogougao ima površinu od A kvadratnih jedinica, obim od P jedinica, apotemu od a jedinica, tada je površina jednaka..."
2,3,average cost for custom canopy,"[Čestitamo! Pronašli ste BuyShade.com, vaš izvor za šatore koji se brzo postavljaju, prekrivače za stolove, prilagođene šatore, štampane zastave i još mnogo toga. Sa stotinama konfiguracija proizvoda i stilova u našem portfoliju i timom vrhunskih dizajnera, specijalizovani smo za pravljenje pril..."
3,4,šta je hardver u računaru,"[Hardver se najbolje opisuje kao uređaj, poput tvrdog diska, koji je fizički povezan sa računarom ili nešto što se može fizički dodirnuti. CD-ROM, monitor računarskog ekrana, štampač i grafička kartica su primeri računarskog hardvera. Hardver služi kao sistem za isporuku softverskih rešenja. Har..."
4,5,edi logistics definition,"[Kao i mnoge druge rane informacione tehnologije, EDI je bio inspirisan razvojem vojne logistike. Složenost berlinske blokade iz 1948. zahtevala je razvoj koncepata i metoda za razmenu, ponekad preko 300 baud teletipskog modema, ogromnih količina podataka i informacija o prevoženim robama. U EDI..."
5,6,Zašto bi trebalo da bude ilegalno rekreativno korišćenje marihuane?,"[Neki se pitaju 'zašto bi marihuana trebalo da bude legalizovana?' ali mi bismo trebali da se zapitamo Zašto bi marihuana trebalo da bude ilegalna?. Sa filozofskog stanovišta, pojedinci zaslužuju pravo da sami donose odluke. Vlada ima pravo da ograniči te odluke samo ako postupci pojedinca ugrož..."
6,7,U koju klasu spadaju paukovi?,"[PODFAKULTET HELICERATA, KLASA ARACHNIDA. Paukovi. Ova grupa sadrži mnoge poznate organizme, uključujući paukove, grinje, škorpije i krpelje. Pogledajte velikog pauka sa desne strane. Ponovo primetite da postoje dva dela tela, glavotučak i abdomen. Na glavotučku se nalaze dva do četiri para jedn..."
7,9,Koliko košta da sterilizujem svoju ženku kunića?,"[Rizik od reproduktivnih karcinoma (jajnika, materice, mlečnih žlezda) kod nekastrirane ženke kunića gotovo je eliminisan sterilizacijom vaše ženke kunića. Vaš kastrirani mužjak kunić takođe će živeti duže, s obzirom na to da neće biti izazvan da se bori sa drugim životinjama (kunićima, mačkama,..."


In [46]:
import pandas as pd
pd.set_option('display.max_colwidth', 300)

nq_path = "datasets/naquestions.parquet"
pd.read_parquet(nq_path, engine='fastparquet')

,id,query,passage_text
0,-5501481664893105662,ko je predložio da elektroni pokazuju svojstva talasa i čestica,"[Kroz rad Maks Planka, Alberta Einštajna, Luj de Broglia, Artura Komptona, Nilsa Bora i mnogih drugih, trenutna naučna teorija tvrdi da sve čestice takođe imaju prirodu talasa (i obrnuto). Ovaj fenomen je potvrđen ne samo za elementarne čestice, već i za složene čestice poput atoma pa čak i mole..."
1,8594030243394572667,Koliko senatora ima u Senatu SAD-a?,"[Senat Sjedinjenih Američkih Država se sastoji od 100 članova, po dva iz svake od 50 saveznih država. U nastavku je lista trenutnih senatora SAD-a koji sede u 115. Kongresu Sjedinjenih Američkih Država.]"
2,-716176363900512091,Faza promene gasa u čvrsto se naziva,"[Depozicija je termodinamički proces, faza prelaska u kojoj gas prelazi u čvrsto stanje bez prolaska kroz tečnu fazu. Suprotno od depozicije je sublimacija i zbog toga se ponekad depozicija naziva desublimacija.]"
3,-5593761806871287545,Koja je najduža engleska reč u rečniku?,"[Najduža reč u bilo kom od glavnih engleskih jezičkih rečnika je pneumonoultramicroscopicsilicovolcanoconiosis, reč koja se odnosi na bolest pluća koja se dobija udisanjem veoma finih čestica silicijuma, specifično sa vulkana; medicinski, isto značenje ima kao silikoza. Reč je namerno skovana ka..."
4,2074579308847994444,koliki procenat Zemljine površine je voda,"[Voda je raspoređena širom Zemlje. Većina vode u Zemljinoj atmosferi i kori potiče iz slane morske vode svetskog okeana, dok slatka voda čini samo 2.5% ukupne količine. Zbog okeana koji pokrivaju otprilike 78% površine Zemlje reflektujući plavu svetlost, Zemlja izgleda plavo iz svemira i često s..."
5,-4418682909598136345,Odakle potiče supa od gnezda ptica?,"[Jesti gnezda ptica su gnezda ptica koja stvaraju ptice lastavice koristeći zgrušanu pljuvačku, koja se bere radi ljudske konzumacije. Posebno su cenjena u kineskoj kulturi zbog svoje retkosti, navodno visoke nutritivne vrednosti i izuzetnog ukusa. Jesti gnezda ptica su među najskupljim životinj..."
6,-4809486218828226485,Odakle dolaze neutroni u nuklearnoj fisiji?,"[Izvor neutrona za pokretanje je izvor neutrona koji se koristi za stabilno i pouzdano pokretanje lančane nuklearne reakcije u nuklearnim reaktorima, kada su punjeni svežim nuklearnim gorivom, čiji neutronski fluks od spontane fisije nije dovoljan za pouzdano pokretanje, ili nakon produženih per..."


In [53]:
from openai import OpenAI
client = OpenAI()
# nq
client.batches.retrieve("batch_XK2ZF1N0f4TjjG6O5SvbmFab")

Batch(id='batch_XK2ZF1N0f4TjjG6O5SvbmFab', completion_window='24h', created_at=1724333607, endpoint='/v1/chat/completions', input_file_id='file-hR5Qe1c0sSYdhJ8ulOKU45cJ', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1724333649, error_file_id=None, errors=None, expired_at=None, expires_at=1724420007, failed_at=None, finalizing_at=1724333649, in_progress_at=1724333607, metadata=None, output_file_id='file-PRJn5nsDIsX8OnedXNuDzzCu', request_counts=BatchRequestCounts(completed=10, failed=0, total=10))

In [54]:
from openai import OpenAI
client = OpenAI()
# ms_marco
client.batches.retrieve("batch_qFbTrX90NiVLCZ9ij4WtOljJ")

Batch(id='batch_qFbTrX90NiVLCZ9ij4WtOljJ', completion_window='24h', created_at=1724333601, endpoint='/v1/chat/completions', input_file_id='file-L8mLrI8xszNDhmjFVBOtn8X5', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1724333675, error_file_id=None, errors=None, expired_at=None, expires_at=1724420001, failed_at=None, finalizing_at=1724333674, in_progress_at=1724333601, metadata=None, output_file_id='file-QhzI3UzzXhcSPcunIY0hdBCb', request_counts=BatchRequestCounts(completed=10, failed=0, total=10))